---
title: "Backtesting Trading Strategy"

description: "Bactest the Stock for RSI and SMA indicator"
author: "Aakash Basnet"
date: "2024/01/26
page-layout: full
categories:
  - python
  - stock
format:
  html:
    code-fold: true
jupyter: python3
---

# Installation
On your terminal run the following command to install yfinance module.
```
pip install backtesting
```

# Extract Company in SP500 
Firstly, lets extract the SP500 tickers from wikipedia table.

In [7]:
import pandas as pd


def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

In [8]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [9]:
from backtesting.test import GOOG

backtest = Backtest(GOOG, System, commission=.002)
backtest.run()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                      2.7933
Equity Final [$]                  10017.44422
Equity Peak [$]                    10978.3801
Return [%]                            0.17444
Buy & Hold Return [%]               703.45824
Return (Ann.) [%]                     0.02045
Volatility (Ann.) [%]                 4.94121
CAGR [%]                               0.0141
Sharpe Ratio                          0.00414
Sortino Ratio                         0.00536
Calmar Ratio                          0.00204
Max. Drawdown [%]                   -10.00745
Avg. Drawdown [%]                    -9.34009
Max. Drawdown Duration     2653 days 00:00:00
Avg. Drawdown Duration     1410 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                        9.68758
Worst Trade [%]                   

In [10]:
backtest.plot()

GridPlot(id='p1873', ...)